<a href="https://colab.research.google.com/github/AlejandroVillazonG/Tareas_INF395/blob/main/T2/desafio_kaggle.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<center><img src="https://www.inf.utfsm.cl/images/slides/Departamento-de-Informtica_HORIZONTAL.png" title="Title text" width= 800 /></center>
<hr style="height:2px;border:none"/>
<H1 align='center'> DESAFÍO TAREA 2</H1>

<H3 align='center'> INF395 2023-2 </H3>
<hr style="height:2px;border:none"/>

## Integrantes:
* Joaquín Aguirre (201910031-9)
* Alejandro Villazón (201910009-2)
* Dominique Yessouroun (201910005-K)

Importamos las librerías necesarias:

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.feature_selection import mutual_info_classif

import os
import json
from sklearn.preprocessing import StandardScaler

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.applications import ResNet50

import multiprocessing

import warnings
# Ignorar las advertencias
warnings.filterwarnings("ignore")

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
path = '/content/drive/MyDrive/10sem 2023-2/INF395/T2/Kaggle/'

## Preprocesar imágenes

In [ ]:
!unzip '{path}inf-395-tarea-2.zip'

In [ ]:
main_path = '/content/'

# with open(main_path + "hierarchy_dict.json", "r") as archivo:
    # subclases = json.load(archivo)

# with open(main_path + "food_categories_dict.json", "r") as archivo:
    # superclases = json.load(archivo)

# df_train = pd.read_csv(main_path+'train.csv')
# df_test = pd.read_csv(main_path+'test.csv')

In [ ]:
def preprocess_image(main_path, path_image, image_size):
    image = keras.utils.load_img(main_path + path_image)
    image = image.resize((image_size, image_size))
    return keras.utils.img_to_array(image)

def preprocess_train_test_set(main_path, paths, image_size):
    return np.array([preprocess_image(main_path, path_image, image_size) for path_image in paths])

In [ ]:
image_size = 50

In [ ]:
X = preprocess_train_test_set(main_path, df_train['path'], image_size)
np.save(path+'X.npy', X)

In [ ]:
X = None
X_test = preprocess_train_test_set(main_path, df_test['path'], image_size)
np.save(path+'X_test.npy', X_test)

# Cargar data

In [4]:
X = np.load(path + 'X.npy')
# X_test = np.load(path + 'X_test.npy')

In [5]:
df_train = pd.read_csv(path+'train.csv')
y = df_train[['food_category', 'dish']]

In [6]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.5, random_state=42)

X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

# Entrenamiento de la NN

In [7]:
def macro_f1_sup(y_true, y_pred):
    """
    Calcula el Macro F1-Score.
    """
    y_pred = tf.one_hot(tf.argmax(y_pred, axis=1), depth=3)

    tp = tf.cast(tf.math.count_nonzero(y_pred * y_true, axis=1), tf.float32)
    fp = tf.cast(tf.math.count_nonzero(y_pred * (1-y_true), axis=1), tf.float32)
    fn = tf.cast(tf.math.count_nonzero((1-y_pred) * y_true, axis=1), tf.float32)

    precision = tp / (tp + fp)
    recall = tp / (tp + fn)

    f1 = 2*precision*recall / (precision + recall)
    f1 = tf.where(tf.math.is_nan(f1), tf.zeros_like(f1), f1)

    return tf.reduce_mean(f1)


def macro_f1_sub(y_true, y_pred):
    """
    Calcula el Macro F1-Score.
    """
    y_pred = tf.one_hot(tf.argmax(y_pred, axis=1), depth=101)

    tp = tf.cast(tf.math.count_nonzero(y_pred * y_true, axis=1), tf.float32)
    fp = tf.cast(tf.math.count_nonzero(y_pred * (1-y_true), axis=1), tf.float32)
    fn = tf.cast(tf.math.count_nonzero((1-y_pred) * y_true, axis=1), tf.float32)

    precision = tp / (tp + fp)
    recall = tp / (tp + fn)

    f1 = 2*precision*recall / (precision + recall)
    f1 = tf.where(tf.math.is_nan(f1), tf.zeros_like(f1), f1)

    return tf.reduce_mean(f1)

In [8]:
image_size = 50

In [9]:
cnn_model = ResNet50(weights='imagenet', include_top=False, input_shape=(image_size, image_size, 3))

for layer in cnn_model.layers:
    layer.trainable = False

model = keras.Sequential()
model.add(cnn_model)
model.add(layers.Flatten())
model.add(layers.Dense(128, activation='relu'))
model.add(layers.Dropout(0.5))

top_level_output = layers.Dense(3, activation='softmax')(model.layers[-1].output)
fine_grained_output = layers.Dense(101, activation='softmax')(model.layers[-1].output)

model = keras.Model(inputs=model.inputs, outputs=[top_level_output, fine_grained_output])

model.summary()

94765736/94765736 [==============================] - 0s 0us/step
Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 resnet50_input (InputLayer  [(None, 50, 50, 3)]          0         []                            
 )                                                                                                
                                                                                                  
 resnet50 (Functional)       (None, 2, 2, 2048)           2358771   ['resnet50_input[0][0]']      
                                                          2                                       
                                                                                                  
 flatten (Flatten)           (None, 8192)                 0         ['resnet50[0][0]']            
                             

In [10]:
model.compile(optimizer='adam', loss=['categorical_crossentropy', 'categorical_crossentropy'],
              metrics={'dense_1': macro_f1_sup, 'dense_2': macro_f1_sub})
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 resnet50_input (InputLayer  [(None, 50, 50, 3)]          0         []                            
 )                                                                                                
                                                                                                  
 resnet50 (Functional)       (None, 2, 2, 2048)           2358771   ['resnet50_input[0][0]']      
                                                          2                                       
                                                                                                  
 flatten (Flatten)           (None, 8192)                 0         ['resnet50[0][0]']            
                                                                                              

In [11]:
y_train_sup = tf.one_hot(y_train['food_category'].values, depth=3)
y_train_sub = tf.one_hot(y_train['dish'].values, depth=101)

y_val_sup = tf.one_hot(y_val['food_category'].values, depth=3)
y_val_sub = tf.one_hot(y_val['dish'].values, depth=101)

In [12]:
E_S = keras.callbacks.EarlyStopping(monitor='val_dense_2_macro_f1_sub', mode='max', patience=20, restore_best_weights=True)

In [13]:
history = model.fit(X_train, [y_train_sup, y_train_sub],
                    batch_size = 256,
                    epochs = 200,
                    callbacks=[E_S],#[checkpoint, custom_checkpoint],
                    verbose = 1,
                    validation_data = (X_val, [y_val_sup, y_val_sub])
                    )

Epoch 1/200
119/119 [==============================] - 36s 142ms/step - loss: 5.9459 - dense_1_loss: 1.1884 - dense_2_loss: 4.7575 - dense_1_macro_f1_sup: 0.7313 - dense_2_macro_f1_sub: 0.0100 - val_loss: 5.6291 - val_dense_1_loss: 1.0136 - val_dense_2_loss: 4.6155 - val_dense_1_macro_f1_sup: 0.7463 - val_dense_2_macro_f1_sub: 0.0100
Epoch 2/200
119/119 [==============================] - 11s 90ms/step - loss: 5.5926 - dense_1_loss: 0.9773 - dense_2_loss: 4.6153 - dense_1_macro_f1_sup: 0.7406 - dense_2_macro_f1_sub: 0.0112 - val_loss: 5.5563 - val_dense_1_loss: 0.9405 - val_dense_2_loss: 4.6158 - val_dense_1_macro_f1_sup: 0.7464 - val_dense_2_macro_f1_sub: 0.0074
Epoch 3/200
119/119 [==============================] - 10s 87ms/step - loss: 5.5288 - dense_1_loss: 0.9139 - dense_2_loss: 4.6149 - dense_1_macro_f1_sup: 0.7401 - dense_2_macro_f1_sub: 0.0110 - val_loss: 5.4940 - val_dense_1_loss: 0.8773 - val_dense_2_loss: 4.6167 - val_dense_1_macro_f1_sup: 0.7436 - val_dense_2_macro_f1_sub: 0

# Predecimos el Test Set

In [14]:
X_test = np.load(path+'X_test.npy')

In [15]:
y_pred = model.predict(X_test, verbose=0)

In [16]:
sample_submission = pd.read_csv(path+'sample_submission.csv')

In [17]:
sample_submission

,path,classes
0,images/caprese_salad/820512.jpg,0 0
1,images/hamburger/2062556.jpg,0 0
2,images/lasagna/2381454.jpg,0 0
3,images/ceviche/1995.jpg,0 0
4,images/poutine/892377.jpg,0 0
...,...,...
25245,images/ravioli/2087725.jpg,0 0
25246,images/ice_cream/504131.jpg,0 0
25247,images/cannoli/1187751.jpg,0 0
25248,images/pancakes/477236.jpg,0 0


In [18]:
sample_submission['sup_clase'] = np.argmax(y_pred[0], axis=1)
sample_submission['sub_clase'] = np.argmax(y_pred[1], axis=1)

In [19]:
sample_submission['classes'] = sample_submission['sup_clase'].astype(str) + ' ' + sample_submission['sub_clase'].astype(str)

In [20]:
sample_submission = sample_submission.drop(['sup_clase', 'sub_clase'], axis = 1)

In [21]:
sample_submission

,path,classes
0,images/caprese_salad/820512.jpg,2 88
1,images/hamburger/2062556.jpg,2 88
2,images/lasagna/2381454.jpg,2 99
3,images/ceviche/1995.jpg,2 88
4,images/poutine/892377.jpg,2 88
...,...,...
25245,images/ravioli/2087725.jpg,2 93
25246,images/ice_cream/504131.jpg,2 88
25247,images/cannoli/1187751.jpg,2 83
25248,images/pancakes/477236.jpg,2 83


In [22]:
sample_submission.to_csv(path+'v1.csv', index=False)

# Guardamos la NN

In [23]:
model.save(path+'v1.keras')